In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
num_data = 100000

In [ ]:
!printf data/**\\n.*\\n*.ipynb\\nregistry.db > .feastignore

In [ ]:
%pip uninstall -y codeflare-sdk

In [ ]:
%pip install 'feast[postgres, redis]' psycopg2

In [ ]:
from numpy import arange, random, round
from numpy.random import seed
from datetime import datetime, timedelta


seed(123456)
addresses = arange(0, num_data)
home_value = random.rand(num_data) * 900000 + 50000

home_sq_ft = random.rand(num_data) * 2300 + 700
num_beds = random.randint(1, 10, num_data)
basement_sq_ft = random.rand(num_data) * 800 + 300

distance_to_nearest_fire_hydrant = random.rand(num_data) * 1400 + 100  # in feet
time_since_last_claim = (random.rand(num_data) * 119 + 1).astype(int)  # in months
distance_from_emergency_systems = random.rand(num_data) * 2.9 + 0.1  # in miles
fire_extinguisher_in_house = random.randint(0, 2, num_data)  # as an int boolean


hoi_monthly_cost = round(
    (random.rand(num_data) * 0.2 + 0.8) *
    (home_value +
     home_sq_ft * 100 +
     num_beds * 27000 +
     basement_sq_ft * 20 +
     distance_to_nearest_fire_hydrant**2 * -5 / 1500 +
     time_since_last_claim * 15 +
     distance_from_emergency_systems * -500 +
     fire_extinguisher_in_house * -73000
    ) / 1000,
    2)
timestamps = [datetime.now() - timedelta(minutes=5 * i) + timedelta(days=30) for i in range(num_data)]

In [ ]:
from pandas import DataFrame

data = DataFrame()
data["address_id"] = addresses
data["home_sq_ft"] = home_sq_ft
data["num_beds"] = num_beds
data["basement_sq_ft"] = basement_sq_ft
data["hoi_monthly_cost"] = hoi_monthly_cost
data["event_timestamp"] = timestamps
data["created"] = timestamps

In [ ]:
from os import getenv
from sqlalchemy import create_engine, engine

connection_string = engine.URL.create(
    drivername="postgresql",
    username=getenv('USER'),
    password=getenv('PASSWORD'),
    host=getenv('HOST'),
    database=getenv('DBNAME'),
)

this_engine = create_engine(connection_string)
data.to_sql('address_values', schema="hoi", con=this_engine, if_exists="append", index=False)

In [ ]:
from psycopg2 import connect

try:
    conn = connect(dbname=getenv('DBNAME'), user=getenv('USER'), host=getenv('HOST'), password=getenv('PASSWORD'))
except:
    print("I am unable to connect to the database")

with conn.cursor() as curs:
    curs.execute("SELECT count(1) from hoi.address_values where event_timestamp < now()")
    single_row = curs.fetchone()
    print(single_row)

conn.close()

In [ ]:
from pandas import DataFrame

data = DataFrame()
data["address_id"] = addresses
data["home_value"] = home_value
data["event_timestamp"] = timestamps
data["created"] = timestamps

In [ ]:
from os import getenv
from sqlalchemy import create_engine, engine

connection_string = engine.URL.create(
    drivername="postgresql",
    username=getenv('USER'),
    password=getenv('PASSWORD'),
    host=getenv('HOST'),
    database=getenv('DBNAME'),
)

this_engine = create_engine(connection_string)
data.to_sql('vendor_address_values', schema="hoi", con=this_engine, if_exists="append", index=False)

In [ ]:
from psycopg2 import connect

try:
    conn = connect(dbname=getenv('DBNAME'), user=getenv('USER'), host=getenv('HOST'), password=getenv('PASSWORD'))
except:
    print("I am unable to connect to the database")

with conn.cursor() as curs:
    curs.execute("SELECT count(1) from hoi.vendor_address_values where event_timestamp < now()")
    single_row = curs.fetchone()
    print(single_row)

conn.close()